In [12]:
import tvm
from tvm import relay
import numpy as np

In [9]:
dshape = [1, 32, 64, 64]
kshape = [32, 32, 3, 3]
dtype = 'int16'
x = relay.var("x", shape=dshape, dtype=dtype)
w = relay.var("w", dtype=dtype)

y = relay.nn.bitserial_conv2d(x, w, channels=32)
func = relay.Function([x, w], y)

data = np.random.uniform(-2, 2, size=dshape).astype(dtype)
kernel = np.random.uniform(-2, 2, size=kshape).astype(dtype)

In [10]:
with relay.build_config(opt_level=3):
    graph, lib, params = relay.build(func, 'llvm', params={'w': kernel})

Cannot find config for target=llvm, workload=('bitserial_conv2d_nchw', (1, 32, 64, 64, 'int16'), (32, 32, 3, 3, 'int16'), (1, 1), (0, 0), 1, 1, 'uint32', 'int16', 1). A fallback configuration is used, which may bring great performance regression.


In [13]:
intrp1 = relay.create_executor('graph', ctx=tvm.cpu(), target='llvm')

In [14]:
op_res1 = intrp1.evaluate(func)(data, kernel)

TVMError: Traceback (most recent call last):
  [bt] (8) /home/tvm/build/libtvm.so(tvm::relay::Expr tvm::relay::TypeInferencer::Resolver::AttachCheckedType<tvm::relay::FunctionNode>(tvm::relay::FunctionNode const*)+0x1eb) [0x7f40cf132a5b]
  [bt] (7) /home/tvm/build/libtvm.so(tvm::relay::ExprMutator::VisitExpr_(tvm::relay::FunctionNode const*)+0x5ef) [0x7f40ceed6fbf]
  [bt] (6) /home/tvm/build/libtvm.so(tvm::relay::ExprMutator::VisitExpr(tvm::relay::Expr const&)+0x9e) [0x7f40ceed7fbe]
  [bt] (5) /home/tvm/build/libtvm.so(tvm::relay::ExprFunctor<tvm::relay::Expr (tvm::relay::Expr const&)>::VisitExpr(tvm::relay::Expr const&)+0xd2) [0x7f40ceedeb02]
  [bt] (4) /home/tvm/build/libtvm.so(std::_Function_handler<tvm::relay::Expr (tvm::NodeRef const&, tvm::relay::ExprFunctor<tvm::relay::Expr (tvm::relay::Expr const&)>*), tvm::relay::ExprFunctor<tvm::relay::Expr (tvm::relay::Expr const&)>::InitVTable()::{lambda(tvm::NodeRef const&, tvm::relay::ExprFunctor<tvm::relay::Expr (tvm::relay::Expr const&)